The master_file is an example of how the address_compare library can be used to load training and test data, tag the addresses, standardize the addresses, and compare the different address lists.  It can serve as a reusable program by updating the input parameters.  If ground truth files are available, it will also show how well the tagger and compare functions perform.

In [2]:
from collections import defaultdict
from collections import OrderedDict
from address_compare import standardizers as stndrdzr
from address_compare import matcher as mtch
from address_compare.tagging import AddressTagger
from address_compare import address_randomizer as add_rndm
import json
import pandas as pd
import sys
import numpy as np
import sklearn

                     address_1                         address_2  match
0        #3672, 84750 E 33 LN         BLDG 3672 - 84750 E 33 LN    True
1        #3672, 84750 E 33 LN       BLDG 3672 - 84750 E 33rd LN    True
2        #3672, 84750 E 33 LN           BLDG 3672-84750 E 33 LN    True
3        #3672, 84750 E 33 LN         BLDG 3672-84750 E 33rd LN    True
4        #3672, 84750 E 33 LN          BLDG 3672, 84750 E 33 LN    True
5        #3672, 84750 E 33 LN        BLDG 3672, 84750 E 33rd LN    True
6        #3672, 84750 E 33 LN           84750 E 33 LN, BLDG 3672   True
7        #3672, 84750 E 33 LN         84750 E 33rd LN, BLDG 3672   True
8        #3672, 84750 E 33 LN              3672 - 84750 E 33 LN    True
9        #3672, 84750 E 33 LN            3672 - 84750 E 33rd LN    True
10  BLDG 3672 - 84750 E 33 LN           BLDG 3672-84750 E 33 LN    True
11  BLDG 3672 - 84750 E 33 LN         BLDG 3672-84750 E 33rd LN    True
12  BLDG 3672 - 84750 E 33 LN          BLDG 3672, 84750 E 33 LN 

Although this notebook is an example of how to use the address_compare library, the following parameters can be changed in order to control the inputs and outputs.  I.e., the following parameters allow this file to be a reusable program sitting on top of the address_compare library.


The **run_mode** variable controls which portions of this notebook are run.  Options are:
- **'tagger'** = run the address tagger against a single file that also contains the ground truths.  output will show how well the tagger did against the ground truths.  tagger will only run against the file found in @file_location_raw_addresses_1
- **'comparer'** = tag 2 separate lists of addresses and find matches between the lists.  no ground truths for comparisons.  no tagger ground truths or match ground truths included. program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2
- **'comparer_truths'** = run the comparer and validate the matcher performance against the ground truths. program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2.  in addition, the matched ground truths will be found in @file_name_ground_truth_matches
- **'all'** = runs all 3 modes.  i.e., tagger results compared against the ground truths and the matcher results compared against the ground truths.  program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2. in addition, the matched ground truths will be found in @file_name_ground_truth_matches

In [3]:
retrain_crf_tagger = False #if True, the specified training file will be used to retrain the CRF Tagger

run_mode = 'comparer_truths' #choose from ['tagger','comparer','comparer_truths','all']


standardize_addresses = True #if True, the tagged address components will be standardized (changed to upper case, unit types, street types, etc. changed to long form names)

use_raw_address_files = True #if False, only the specified number of randomly created addresses above will be used
num_rndm_addresses_to_create = 100 #if use_raw_address_files = False, the number of addresses that will be randomly created for use in the tagger and compare functions

field_name_raw_addresses = 'Single String Address' #represents the name of the field in the raw address files containing the raw address (street information)
field_name_record_id = 'Record_ID' #represents the name of the field containing the Record ID in the raw files; if not present in the raw files, populate with None

#file_location_raw_addresses_1 = 'data\\standardized tagged washington state addresses.xlsx'
#file_location_raw_addresses_1 = 'data\\tagged standardized colorado Stores.xlsx'
file_location_raw_addresses_1 = 'data\\MarijuanaApplicants - test data list 1.xlsx'
file_location_raw_addresses_2 = 'data\\MarijuanaApplicants - test data list 2.xlsx'

file_name_ground_truth_matches = 'data\\marijuana applicants test data - correct matches.xlsx'

In [4]:
# Placeholder for reading/calling the training data for the CRF Tagger and sending the training data to train the model
if retrain_crf_tagger:
    with open('data/tagged_addresses.json') as f:
        td = json.load(f)
    
    #send training data to CRF tagger to train the model here...

In [5]:
def tagger_vs_ground_truths (file, file_num, field_rec_id, field_raw_address, to_standardize):
    test_file = pd.read_excel(file, keep_default_na=False, dtype=str)
    test_file = stndrdzr.record_id_addition(test_file, field_rec_id)
    
    # Add Empty Missing Columns to Dataframe
    missing_columns = ['CITY','STATE','ZIP_CODE','UNKNOWN']
    test_file = stndrdzr.empty_column_addition(test_file, missing_columns)
    
    tagger = AddressTagger()
    tagged_test_file = tagger.series_to_address_df(test_file[field_raw_address], standardize = to_standardize)
    crf_tagged_test_file = tagged_test_file.join(test_file['Record_ID'])
    
    ground_truth_columns = ['Record_ID', 'Tagged Street Number','Tagged Pre Street Direction','Tagged Street Name','Tagged Street Type','Tagged Post Street Direction','Tagged Unit Type','Tagged Unit Number']
    
    manual_tagged_test_file = test_file[ground_truth_columns].copy()
    manual_tagged_test_file = manual_tagged_test_file.rename(columns={'Tagged Street Number':'STREET_NUMBER',
                                                                     'Tagged Pre Street Direction':'PRE_DIRECTION',
                                                                     'Tagged Street Name':'STREET_NAME',
                                                                     'Tagged Street Type':'STREET_TYPE',
                                                                     'Tagged Post Street Direction':'POST_DIRECTION',
                                                                     'Tagged Unit Type':'UNIT_TYPE',
                                                                     'Tagged Unit Number':'UNIT_NUMBER'})

    cols_for_matcher = ['UNIT_TYPE','UNIT_NUMBER','STREET_NUMBER','PRE_DIRECTION','STREET_NAME','STREET_TYPE','POST_DIRECTION']
    correctly_tagged_addresses = mtch.exact_matcher(crf_tagged_test_file, manual_tagged_test_file, cols_for_matcher)
    
    incorrectly_tagged_addresses = crf_tagged_test_file.mask(crf_tagged_test_file.Record_ID.isin(correctly_tagged_addresses['Record_ID_list_1'])).dropna()
    incorrectly_tagged_addresses = incorrectly_tagged_addresses.merge(test_file[ground_truth_columns], on='Record_ID')

    total_records = crf_tagged_test_file.shape[0]
    correctly_tagged = correctly_tagged_addresses.shape[0]
    incorrectly_tagged = incorrectly_tagged_addresses.shape[0]
    
    tagger_accuracy = correctly_tagged / total_records
    
    metrics_dict = OrderedDict()
    precision_dict = OrderedDict()
    recall_dict = OrderedDict()
    fscore_dict = OrderedDict()
    overallacc_dict = OrderedDict()
    
    for col in cols_for_matcher:
        precision, recall, fscore, ignore = sklearn.metrics.precision_recall_fscore_support(manual_tagged_test_file[col], crf_tagged_test_file[col], average='micro')
        precision_dict[col] = precision
        recall_dict[col] = recall
        fscore_dict[col] = fscore
        overallacc_dict[col] = None
    
    precision_dict['overall_accuracy'] = None
    recall_dict['overall_accuracy'] = None
    fscore_dict['overall_accuracy'] = None
    overallacc_dict['overall_accuracy'] = tagger_accuracy  
    
    metrics_dict['precision'] = precision_dict
    metrics_dict['recall'] = recall_dict
    metrics_dict['fscore'] = fscore_dict
    metrics_dict['overall_accuracy'] = overallacc_dict
    
    metrics_df = pd.DataFrame(metrics_dict)
    
    # Dictionary of DataFrames for Excel Tagger Test File
    dataframes_for_tagger_excel = {'test_data_file': test_file, 
                                   'crf_tagger_output': crf_tagged_test_file,
                                   'ground_truth_test_file': manual_tagged_test_file, 
                                   'correctly_tagged': correctly_tagged_addresses,
                                   'incorrectly_tagged': incorrectly_tagged_addresses,
                                  'tagger_metrics': metrics_df}
    
    # Write Dict of DataFrames to Excel
    output_name = 'output\\file ' + str(file_num) + ' tagger accuracy output.xlsx'
    tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
    for sheet, frame in  dataframes_for_tagger_excel.items():
        frame.to_excel(tagger_writer, sheet_name = sheet)
    tagger_writer.save()
    
    return test_file, tagged_test_file

In [6]:
if run_mode == 'tagger':
    raw_address_list_1, tagged_address_list_1 = tagger_vs_ground_truths(file_location_raw_addresses_1, 1, field_name_record_id, field_name_raw_addresses, standardize_addresses)
    sys.exit()
elif run_mode == 'all':
    raw_address_list_1, tagged_address_list_1 = tagger_vs_ground_truths(file_location_raw_addresses_1, 1, field_name_record_id, field_name_raw_addresses, standardize_addresses)
    raw_address_list_2, tagged_address_list_2 = tagger_vs_ground_truths(file_location_raw_addresses_2, 2, field_name_record_id, field_name_raw_addresses, standardize_addresses)

In [7]:
# Placeholder for reading/calling the 2 lists of raw addresses
if run_mode in ['comparer','comparer_truths']:
    if use_raw_address_files:
        raw_address_list_1 = pd.read_excel(file_location_raw_addresses_1)
        raw_address_list_2 = pd.read_excel(file_location_raw_addresses_2)
    else:
        raw_address_list_1 = add_rndm.random_addresses(num_rndm_addresses_to_create, field_name_raw_addresses)
        raw_address_list_2 = add_rndm.random_addresses(num_rndm_addresses_to_create, field_name_raw_addresses)

In [8]:
# Add a field called Record_ID if it doesn't already exist in the raw address files
if run_mode in ['comparer','comparer_truths']:
    raw_address_list_1 = stndrdzr.record_id_addition(raw_address_list_1, field_name_record_id)
    raw_address_list_2 = stndrdzr.record_id_addition(raw_address_list_2, field_name_record_id)

In [9]:
# Add Empty Missing Columns to Dataframe
if run_mode in ['comparer','comparer_truths']:
    missing_columns = ['CITY','STATE','ZIP_CODE','UNKNOWN']
    raw_address_list_1 = stndrdzr.empty_column_addition(raw_address_list_1, missing_columns)
    raw_address_list_2 = stndrdzr.empty_column_addition(raw_address_list_2, missing_columns)

In [10]:
# Call the trained CRF Tagger on the 2 lists of raw addresses
if run_mode in ['comparer','comparer_truths']:
    at = AddressTagger()
    tagged_address_list_1 = at.series_to_address_df(raw_address_list_1[field_name_raw_addresses], standardize = standardize_addresses)
    tagged_address_list_2 = at.series_to_address_df(raw_address_list_2[field_name_raw_addresses], standardize = standardize_addresses)

In [11]:
# Check for Errors in Zip Codes and Replace City Names with Primary City from Zip Code
raw_address_list_1 = stndrdzr.fix_cities_zips(raw_address_list_1)
raw_address_list_2 = stndrdzr.fix_cities_zips(raw_address_list_2)

In [12]:
# Add Remaining Columns from Raw Address Dataframes to Tagged Address Dataframes
joined_address_list_1 = tagged_address_list_1.join(raw_address_list_1[['Record_ID','CITY','STATE','ZIP_CODE','UNKNOWN','Zip_Code_Error']])
joined_address_list_2 = tagged_address_list_2.join(raw_address_list_2[['Record_ID','CITY','STATE','ZIP_CODE','UNKNOWN','Zip_Code_Error']])

In [13]:
# Remove Addresses with Zip Code Errors (I.e., where the Zip Code is not valid for the given state)
error_addresses_list_1 = joined_address_list_1.where(joined_address_list_1.Zip_Code_Error == "Yes").dropna()
error_addresses_list_2 = joined_address_list_2.where(joined_address_list_2.Zip_Code_Error == "Yes").dropna()

In [14]:
# Only Addresses without Zip Code Errors (I.e., where the Zip Code is valid for the given state)
nonerror_addresses_list_1 = joined_address_list_1.where(joined_address_list_1.Zip_Code_Error == "No").dropna()
nonerror_addresses_list_2 = joined_address_list_2.where(joined_address_list_2.Zip_Code_Error == "No").dropna()

In [15]:
nonerror_addresses_list_1 = nonerror_addresses_list_1.astype({'Record_ID':'int', 'ZIP_CODE':'int'})
nonerror_addresses_list_2 = nonerror_addresses_list_2.astype({'Record_ID':'int', 'ZIP_CODE':'int'})
nonerror_addresses_list_1 = nonerror_addresses_list_1.astype({'Record_ID':'str', 'ZIP_CODE':'str'})
nonerror_addresses_list_2 = nonerror_addresses_list_2.astype({'Record_ID':'str', 'ZIP_CODE':'str'})

In [16]:
# Intra-Grouping of Tagged Address Lists to Consolidate Duplicates
if run_mode == 'comparer':
    grouped_address_list_1 = stndrdzr.consolidate_address_list(nonerror_addresses_list_1)
    grouped_address_list_2 = stndrdzr.consolidate_address_list(nonerror_addresses_list_2)
else:
    grouped_address_list_1 = nonerror_addresses_list_1.copy()
    grouped_address_list_2 = nonerror_addresses_list_2.copy()

In [17]:
# Call Either the Exact Match or Learning Match Functions to match the 2 lists
exact_matches = mtch.exact_matcher(grouped_address_list_1, grouped_address_list_2)

In [18]:
unmatched_address_list_1 = grouped_address_list_1.mask(grouped_address_list_1.Record_ID.isin(exact_matches['Record_ID_list_1'])).dropna()
unmatched_address_list_2 = grouped_address_list_2.mask(grouped_address_list_2.Record_ID.isin(exact_matches['Record_ID_list_2'])).dropna()

In [19]:
# Dictionary of DataFrames for Excel File
dataframes_for_excel = {'raw_addresses_list_1': raw_address_list_1, 'raw_addresses_list2': raw_address_list_2,
                        'zip_errors_list1': error_addresses_list_1, 'zip_errors_list2': error_addresses_list_2,
                       'exact_matches': exact_matches, 'unmatched_list_1': unmatched_address_list_1,
                       'unmatched_list_2': unmatched_address_list_2}

In [20]:
# Write Dict of DataFrames to Excel
writer = pd.ExcelWriter('output\\raw_to_matched_addresses.xlsx', engine='xlsxwriter')
for sheet, frame in  dataframes_for_excel.items():
    frame.to_excel(writer, sheet_name = sheet)
writer.save()

In [21]:
# Compare Output of Exact Match to Manually Tagged Matches
if run_mode in ['comparer_truths','all']:
    manual_matches = pd.read_excel(file_name_ground_truth_matches, dtype=str)
    golden_exact_matches = manual_matches.where(manual_matches.Match_Type.isin(["Exact","Standardized Exact"])).dropna().reset_index()


In [22]:
if run_mode in ['comparer_truths','all']:
    join_cols = ['Record_ID_list_1','Record_ID_list_2']
    subset_columns_exact_matches = exact_matches[join_cols].copy()
    subset_columns_exact_matches['row_index'] = subset_columns_exact_matches.index
    subset_cols_golden_exact_matches = golden_exact_matches[join_cols].copy()
    subset_cols_golden_exact_matches['row_index'] = subset_cols_golden_exact_matches.index
    
    subset_columns_exact_matches = subset_columns_exact_matches.astype(str)
    subset_cols_golden_exact_matches = subset_cols_golden_exact_matches.astype(str)
    
    test_vs_golden_compare = mtch.exact_matcher(subset_columns_exact_matches, subset_cols_golden_exact_matches, join_cols)
    print (test_vs_golden_compare)

    Record_ID_list_1 Record_ID_list_2 row_index_list_1 row_index_list_2
0                  1                1                0                0
1                  3                2                1                1
2                  4                3                2                2
3                  5                4                3                3
4                  6                5                4                4
5                  7                6                5                5
6                  8                7                6                6
7                 11                8                7                7
8                 12                9                8                8
9                 13               10                9                9
10                14               12               10               10
11                15               13               11               11
12                16               14               12          

In [23]:
if run_mode in ['comparer_truths','all']:
    missing_golden_matches = subset_cols_golden_exact_matches.mask(subset_cols_golden_exact_matches.row_index.isin(test_vs_golden_compare.row_index_list_2)).dropna()
    print (missing_golden_matches)

   Record_ID_list_1 Record_ID_list_2 row_index
89              105              303        89


In [24]:
if run_mode in ['comparer_truths','all']:
    matches_not_in_golden = subset_columns_exact_matches.mask(subset_columns_exact_matches.row_index.isin(test_vs_golden_compare.row_index_list_1)).dropna()
    print (matches_not_in_golden)

Empty DataFrame
Columns: [Record_ID_list_1, Record_ID_list_2, row_index]
Index: []


In [25]:
if run_mode in ['comparer_truths','all']:
    total_records_list_1 = raw_address_list_1.shape[0]
    total_records_list_2 = raw_address_list_2.shape[0]
    total_modeled_matches = exact_matches.shape[0]
    total_manual_exact_matches = golden_exact_matches.shape[0]
    total_correct_positive_matches = test_vs_golden_compare.shape[0]
    false_negatives = missing_golden_matches.shape[0]
    false_positives = matches_not_in_golden.shape[0]
    
    accuracy_list_1 = (total_records_list_1 - (false_negatives + false_positives)) / total_records_list_1
    accuracy_list_2 = (total_records_list_2 - (false_negatives + false_positives)) / total_records_list_2
    precision = total_correct_positive_matches / (total_correct_positive_matches + false_positives)
    recall = total_correct_positive_matches / (total_correct_positive_matches + false_negatives)
    
    print ('list 1 accuracy = ', accuracy_list_1)
    print ('list 2 accuracy = ', accuracy_list_2)
    print ('precision = ', precision)
    print ('recall = ', recall)

list 1 accuracy =  0.9986577181208054
list 2 accuracy =  0.9984423676012462
precision =  1.0
recall =  0.998371335504886


In [109]:
# Run Single File against Tagger and Calculate Accuracy
#if run_mode == 'tagger':
#    test_file = pd.read_excel(file_location_raw_addresses_1, keep_default_na=False, dtype=str)
#    test_file = stndrdzr.record_id_addition(test_file, field_name_record_id)
    
#    tagger = AddressTagger()
#    tagged_test_file = tagger.series_to_address_df(test_file[field_name_raw_addresses], standardize = standardize_addresses)
#    crf_tagged_test_file = tagged_test_file.join(test_file['Record_ID'])
    
#    ground_truth_columns = ['Record_ID', 'Tagged Street Number','Tagged Pre Street Direction','Tagged Street Name','Tagged Street Type','Tagged Post Street Direction','Tagged Unit Type','Tagged Unit Number']
    
#    manual_tagged_test_file = test_file[ground_truth_columns].copy()
#    manual_tagged_test_file = manual_tagged_test_file.rename(columns={'Tagged Street Number':'STREET_NUMBER',
#                                                                     'Tagged Pre Street Direction':'PRE_DIRECTION',
#                                                                     'Tagged Street Name':'STREET_NAME',
#                                                                     'Tagged Street Type':'STREET_TYPE',
#                                                                     'Tagged Post Street Direction':'POST_DIRECTION',
#                                                                     'Tagged Unit Type':'UNIT_TYPE',
#                                                                     'Tagged Unit Number':'UNIT_NUMBER'})

#    cols_for_matcher = ['UNIT_TYPE','UNIT_NUMBER','STREET_NUMBER','PRE_DIRECTION','STREET_NAME','STREET_TYPE','POST_DIRECTION']
#    correctly_tagged_addresses = mtch.exact_matcher(crf_tagged_test_file, manual_tagged_test_file, cols_for_matcher)
    
#    incorrectly_tagged_addresses = crf_tagged_test_file.mask(crf_tagged_test_file.Record_ID.isin(correctly_tagged_addresses['Record_ID_list_1'])).dropna()
#    incorrectly_tagged_addresses = incorrectly_tagged_addresses.merge(test_file[ground_truth_columns], on='Record_ID')

#    total_records = crf_tagged_test_file.shape[0]
#    correctly_tagged = correctly_tagged_addresses.shape[0]
#    incorrectly_tagged = incorrectly_tagged_addresses.shape[0]
    
#    tagger_accuracy = correctly_tagged / total_records
#    print ('tagger accuracy = ', tagger_accuracy)
    
#    for col in cols_for_matcher:
#        precision, recall, fscore, ignore = sklearn.metrics.precision_recall_fscore_support(manual_tagged_test_file[col], crf_tagged_test_file[col], average='micro')
#        print ('column = ', col, 'precision = ', precision, 'recall = ', recall, 'f1score = ',fscore)
    
    # Dictionary of DataFrames for Excel Tagger Test File
#    dataframes_for_tagger_excel = {'test_data_file': test_file, 
#                                   'crf_tagger_output': crf_tagged_test_file,
#                                   'ground_truth_test_file': manual_tagged_test_file, 
#                                   'correctly_tagged': correctly_tagged_addresses,
#                                   'incorrectly_tagged': incorrectly_tagged_addresses}
    
#    # Write Dict of DataFrames to Excel
#    tagger_writer = pd.ExcelWriter('output\\tagger accuracy output.xlsx', engine='xlsxwriter')
#    for sheet, frame in  dataframes_for_tagger_excel.items():
#        frame.to_excel(tagger_writer, sheet_name = sheet)
#    tagger_writer.save()
    
#    sys.exit()